# 1-Importaciones

In [55]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 2-Cargar los Datos

In [56]:
df = pd.read_csv('/workspaces/Elreno23-machine-learning-python-template/data/raw/02-linear-regression-medical_insurance_cost.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# 3-Exploracion Inicial

In [57]:
df.info()
df.shape
df.isna().sum()
df.describe().T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


,count,mean,std,min,25%,50%,75%,max
age,1338.0,39.207025,14.049960,18.0000,27.00000,39.000,51.000000,64.00000
bmi,1338.0,30.663397,6.098187,15.9600,26.29625,30.400,34.693750,53.13000
children,1338.0,1.094918,1.205493,0.0000,0.00000,1.000,2.000000,5.00000
charges,1338.0,13270.422265,12110.011237,1121.8739,4740.28715,9382.033,16639.912515,63770.42801


## Conclusiones

La mayoría de los asegurados tiene entre 0 y 2 hijos, el IMC promedio indica sobrepeso, y el coste del seguro (charges) está muy disperso.

# 4-Limpieza de Datos(Inicial)

In [58]:
df = df.drop_duplicates() #Eliminamos duplicados

#iteramos osbre las columnas numericas ignorando la variable objetivo y children(queremos saber si la persona no tiene hijos)
cols_to_check = [col for col in df.select_dtypes(include="number").columns if col not in ["children","charges"]]

#Actualizamos el df eliminando los valores en 0 de estas columnas.
df = df[(df[cols_to_check] > 0).all(axis=1)]

#Categorizamos las columnas(categoricas) obteniendo la cantidad de valores unicos, es decir, cuantas regiones unicas hay?: 4
count_of_categories = {col: df[col].nunique() for col in df.select_dtypes(include="object").columns}
count_of_categories

#Categorizamos las columnas(categoricas) obteniendo sus valores unicos, es decir, cuantos sexos hay?: "female","Male"
unique_values_categories = {col: df[col].unique() for col in df.select_dtypes(include="object").columns}
unique_values_categories



{'sex': array(['female', 'male'], dtype=object),
 'smoker': array(['yes', 'no'], dtype=object),
 'region': array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)}

## Conclusiones

Eliminamos duplicados. Tambien iteramos osbre las columnas numericas excluyendo nuestra variable objetivo y children(porque no queremos eliminar los datos en 0 de esta columna). Creamos un diccionario con la cantidades unicas de las columnas categoricas y hacemos lo mismo pero obteniendo los valores reales(str, bool, etc)


# 8-CEEP
#### Convertir todas las variables en numéricas y escalar el conjunto completo sin dividir previamente en entrenamiento y prueba.




## 8.1
##### Codificación numérica de variables categóricas:

In [59]:
df["sex_n"] = pd.factorize(df["sex"])[0]
df["smoker_n"] = pd.factorize(df["smoker"])[0]
df["region_n"] = pd.factorize(df["region"])[0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1337 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1337 non-null   int64  
 1   sex       1337 non-null   object 
 2   bmi       1337 non-null   float64
 3   children  1337 non-null   int64  
 4   smoker    1337 non-null   object 
 5   region    1337 non-null   object 
 6   charges   1337 non-null   float64
 7   sex_n     1337 non-null   int64  
 8   smoker_n  1337 non-null   int64  
 9   region_n  1337 non-null   int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 114.9+ KB


## Conclusiones

Codificamos y asignamos las variables categoricas a nuevas columnas sin eliminar las originales, esto nos permite revisar facilmente que categorias corresponde a cada numero, interpretar mejor los resultados, visualizarlos y si, mas adelante queremos codificar con otro metodo  ya tenemos las columans originales

## 8.2
#####  Selección de variables numéricas para escalar:

In [60]:
num_variables = [col for col in df.columns if col not in ["sex","smoker","region"]]

## Conclusiones

Elegimos nuestras variables predictoras para escalar(numericas) ignorando las categoricas originales pero incluyendo sus versiones ya codificadas

## 8.3
##### Escalado con MinMaxScaler

In [61]:
scaler = MinMaxScaler() #Inicializamos la iinstancia del Scaler
scal_cols = scaler.fit_transform(df[num_variables]) #Aprende(min y max) y transforma(0 y 1) el rango establecido por el Scaler
df_scal = pd.DataFrame(scal_cols, index=df.index, columns=num_variables) #Creamos un df a partir de esa transformacion(datos escalados)
df_scal

,age,bmi,children,charges,sex_n,smoker_n,region_n
0,0.021739,0.321227,0.0,0.251611,0.0,0.0,0.000000
1,0.000000,0.479150,0.2,0.009636,1.0,1.0,0.333333
2,0.217391,0.458434,0.6,0.053115,1.0,1.0,0.333333
3,0.326087,0.181464,0.0,0.333010,1.0,1.0,0.666667
4,0.304348,0.347592,0.0,0.043816,1.0,1.0,0.666667
...,...,...,...,...,...,...,...
1333,0.695652,0.403820,0.6,0.151299,1.0,1.0,0.666667
1334,0.000000,0.429379,0.0,0.017305,0.0,1.0,1.000000
1335,0.000000,0.562012,0.0,0.008108,0.0,1.0,0.333333
1336,0.065217,0.264730,0.0,0.014144,0.0,1.0,0.000000


## Conclusiones

Aplicamos MinMaxScaler a las variables predictoras numericas y categoricas ya codificcadas, con esto llevamos sus valores a un rango entre 0 y 1, fit_tranform aprendió los valores min y max y luego los transformó a 0 y 1, Este resultado lo almacenamos en un nuevo DF

## 8.4

##### Separar X e y

In [62]:
X = df_scal.drop("charges", axis=1) #Todas las columnas menos charges
y = df_scal["charges"] #Variable objetivo

## Conlusiones

Podemos decir que existen dos metodos para realizar este ejercicio(robusto y simple, asi los llamo yo). En este caso aplicamos el enfoque simple donde el flujo cambia un poco y utlizamos factorize() para codificar nuestras variables. Separar X e y en el enfoque simple se realiza despues del escalado ya que trabajamos sobre el DF completo que contiene las variables codificadas y normalizadas(MinMaxScaler). En ambos enfoques el objetivo es el mismo, definir que columnas se usarán para X y cual como objetivo(y)


## 8.5
##### Division en Entrenamiento y Prueba

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train,X_test, y_train, y_test

(           age       bmi  children  sex_n  smoker_n  region_n
 1114  0.108696  0.230024       0.0    1.0       1.0  1.000000
 968   0.065217  0.263250       0.4    1.0       1.0  1.000000
 599   0.739130  0.580172       0.4    0.0       1.0  0.666667
 170   0.978261  0.686306       0.0    1.0       1.0  0.333333
 275   0.630435  0.286252       0.4    0.0       1.0  1.000000
 ...        ...       ...       ...    ...       ...       ...
 1096  0.717391  0.511165       0.4    0.0       0.0  1.000000
 1131  0.195652  0.805488       0.4    1.0       1.0  0.000000
 1295  0.043478  0.162497       0.2    1.0       1.0  0.000000
 861   0.434783  0.323917       0.6    0.0       1.0  0.000000
 1127  0.369565  0.535378       0.4    0.0       1.0  0.333333
 
 [1069 rows x 6 columns],
            age       bmi  children  sex_n  smoker_n  region_n
 900   0.673913  0.176352       0.0    1.0       1.0  1.000000
 1064  0.239130  0.259349       0.8    0.0       1.0  0.000000
 1256  0.717391  0.549502  

## Conclusiones

Dividimos el df en dos subconjuntos, el 80% de los datos para entrenar el modelo(X_train, y_train) y el 20% restante para testearlo(X_test, y_test)

# 8.6
##### Seleccion de caracteristicas(opcional)

In [64]:
selection_model = SelectKBest(f_regression, k=4) 
selection_model.fit(X_train, y_train)

selected_columns = X_train.columns[selection_model.get_support()]

X_train_sel = pd.DataFrame(selection_model.transform(X_train), columns=selected_columns)
X_test_sel = pd.DataFrame(selection_model.transform(X_test), columns=selected_columns)
X_train_sel.head()

,age,bmi,children,smoker_n
0,0.108696,0.230024,0.0,1.0
1,0.065217,0.263250,0.4,1.0
2,0.739130,0.580172,0.4,1.0
3,0.978261,0.686306,0.0,1.0
4,0.630435,0.286252,0.4,1.0


## Conclusiones

Creamos un modelo de seleccion(SeelectKBest) que va a elegir las 4 cracateisticas mas relevantes(f_regression, k=4). Lo entrenamos y luego rretiornamos un array con las 4 columnas mas relevantes y a partir de ahi creamos dos df con las 4 cracateristicas para los dos conjuntos de pruebas.

# 1-Entrenar y optimizar el modelo

In [65]:
model = LinearRegression()
model.fit(X_train_sel, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


## Conclusiones

En este punto podemos agregar que inicializamos una instancia del modelo LinearRegression y lo entrenamos con el método fit pasandole como parametros el df con nuestras 4 variables de entrenamiento mas relevantes ya escaladas y nuestra variable objetivo

### 1.1 Interpretar los parametros del modelo

In [66]:
print(f"Intercepto (a): {model.intercept_}")
print(f"Coeficientes (b1,b2):{model.coef_}")
X_train_sel


Intercepto (a): 0.3195827183308946
Coeficientes (b1,b2):[ 0.1829699   0.18111814  0.04293572 -0.36780589]


,age,bmi,children,smoker_n
0,0.108696,0.230024,0.0,1.0
1,0.065217,0.263250,0.4,1.0
2,0.739130,0.580172,0.4,1.0
3,0.978261,0.686306,0.0,1.0
4,0.630435,0.286252,0.4,1.0
...,...,...,...,...
1064,0.717391,0.511165,0.4,0.0
1065,0.195652,0.805488,0.4,1.0
1066,0.043478,0.162497,0.2,1.0
1067,0.434783,0.323917,0.6,1.0


## Conclusiones
-Intercepto: coste base que el modelo estima para alguien con los valores minimos en todas las variables seleccionadas.

age: +0.183 -> A mayor edad mayor coste predicho
bmi: +0.181 -> A mayor IMC mayor coste predicho
children: +0.043 -> Mas hijos aumenta ligeramente el coste predicho
smoker_n: -0.368 -> No fumar reduce el coste predicho

Recordemos que factorize codifica por orden de aparición, es decir, en el caso de smoker_n: "yes","no". "yes" = 0 y "no" = 1

### 1.2-Predecir sobre el conjunto de prueba

In [67]:
y_pred = model.predict(X_test_sel) #Probamos el modelo con los datos de entrenamiento
y_pred

array([ 0.10702315,  0.07685187,  0.2083241 ,  0.48685684,  0.12995459,
        0.19726353,  0.45755859,  0.00832554,  0.1556988 ,  0.16287349,
        0.14781084,  0.51037008,  0.4699569 ,  0.25944432,  0.14442757,
        0.13842465,  0.05134994,  0.49351746,  0.03848664,  0.0705325 ,
        0.04276279,  0.45855586,  0.22245537,  0.47182868,  0.47460549,
        0.06698449,  0.55116209,  0.5686984 ,  0.15588314,  0.20127518,
        0.0770878 ,  0.19051001, -0.00377298,  0.17967676,  0.61602286,
        0.18050297,  0.06028104,  0.0447116 ,  0.48287007,  0.13064786,
        0.08387671,  0.45610213,  0.54228697,  0.17124485,  0.09976616,
        0.03950023,  0.07028081,  0.12772953,  0.04874519,  0.13229562,
        0.09175793,  0.1653832 ,  0.47297483,  0.04437078,  0.16108797,
        0.14414371,  0.14902535,  0.02014774,  0.48426463,  0.13172362,
        0.2337337 ,  0.11691   ,  0.18103216,  0.00130779,  0.25281818,
        0.14612565,  0.14268127,  0.47671561,  0.38076578,  0.23

### 1.3 - Evaluar el rendimiento del modelo

In [68]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Error cuadrático medio (MSE): {mse:.4f}")
print(f"Coeficiente de determinación (R²): {r2:.4f}")

Error cuadrático medio (MSE): 0.0092
Coeficiente de determinación (R²): 0.8046


## Conclusiones

MSE(Error cuadratico medio): Medida de cuanto se equivoca el modelo

r2(Coeficiente de determinación): Que tan bien se ajusta el modelo a los datos reales?

Los datos estan escalados entre 0 y 1 por lo tanto los valores mse y r2 reflejan ese rango.
El modelo predice con bastante precisión